In [156]:
import numpy as np
import math as m
import scipy as sp
import matplotlib.pyplot as plt

First we implement a class to represent a normal random variable.

In [104]:
class NormalRV:
    """
    Class for a multivariate normal random variable.
    """

    def __init__(self, mu, Sigma):
        """
        Constructor. Sets:
        - mu (np.array): mean
        - Sigma (np.matrix): covariance matrix
        - S (np.matrix): inverse of Sigma
        - m (np.array): S@mu
        """
        self.mu = mu
        self.Sigma = Sigma
        self.S = np.linalg.inv(Sigma)
        self.m = self.S@self.mu
    
    def pdf(self, x):
        """
        Evaluates the pdf at x. x can be a vector or a matrix; in the latter case, 
        the pdf is evaluated column-wise.

        Args:
        - x (np.array): point(s) at which to evaluate the pdf

        Returns:
        - pdf (np.array): pdf evaluated at x
        """

        mu = self.mu
        Sigma = self.Sigma
        d = len(mu)
        x = np.asarray(x)
        if x.ndim == 1:
            x = x[:, np.newaxis]
        n = x.shape[1]
        pdf = np.zeros(n)
        for i in range(n):
            pdf[i] = 1/np.sqrt((2*np.pi)**d * np.linalg.det(Sigma)) * \
                np.exp(-1/2 * (x[:,i] - mu).T @ np.linalg.inv(Sigma) @ (x[:,i] - mu))
        return pdf

    def sample(self, n):
        """
        Draws n samples from the distribution.

        Args:
        - n (int): number of samples to draw

        Returns:
        - samples (np.array): samples drawn from the distribution
        """

        mu = self.mu
        Sigma = self.Sigma
        return np.random.multivariate_normal(mu, Sigma, n).T
    
    def update_vals(self, m, S):
        """
        Updates the mean and covariance matrix of the distribution.

        Args:
        - m (np.array): new mean
        - S (np.matrix): new covariance matrix

        Sets:
        - mu (np.array): mean
        - Sigma (np.matrix): covariance matrix
        - S (np.matrix): inverse of Sigma
        - m (np.array): S@mu
        """

        self.m = m
        self.S = S
        self.Sigma = np.linalg.inv(S)
        self.mu = self.Sigma@m
        

Note that our update step is:

$$
\theta_{n+1}\leftarrow \theta_n - \alpha \nabla \rho(\theta)
$$

In our case, as both distributions are normal, we can use the fundamental representation of the normal distribution so that its pdf is in the exponential family. This means that:

$$
\nabla \rho(\theta) = \int_{\mathbb{R}^n}-\frac{\pi^2(x)}{q^2_\theta(x)}\nabla\log q_\theta(x) dx
$$

Then, as $\theta = (\mu, \Sigma)$ and using the fundamental representation of the normal distribution we arrive to:

$$
\begin{align*}
\frac{\partial \log}{\partial m}(x) &= x-S^{-1}m = x-\mu\\
\frac{\partial \log}{\partial S}(x) &= -\frac{1}{2}(xx^\top - S^{-1}mm^\top S^{-1} - S^{-1}) = -\frac{1}{2}(xx^\top - \mu\mu^\top - \Sigma)
\end{align*}
$$


We now implement functions to compute the gradient descent update, using the equations above:

In [106]:
def partial_lq_m(x, p):
    """
    Computes the partial derivative of the log-likelihood function with respect to the mean.

    Args:
    - x (np.array): point at which to evaluate the partial derivative
    - p (NormalRV): distribution

    Returns:
    - partial (np.array): partial derivative of the log-likelihood function with respect to the mean
    """

    partial = x - np.reshape(p.mu, (len(p.mu), 1))
    return partial

def partial_lq_S(x, p):
    """
    Computes the partial derivative of the log-likelihood function with respect to the covariance matrix.

    Args:
    - x (np.array): point at which to evaluate the partial derivative
    - p (NormalRV): distribution

    Returns:
    - res (np.array): partial derivative of the log-likelihood function with respect to the covariance matrix
    """

    x = np.asarray(x)
    if x.ndim == 1:
        x = x[:, np.newaxis]
    n = x.shape[1]
    d = len(p.mu)
    res = np.zeros((d,d,n))
    for i in range(n):
        xi = x[:,i]
        res[:,:,i] = -0.5*(np.outer(xi, xi) - np.outer(p.mu, p.mu)- p.Sigma)
    return res


In [107]:
def is_pd(M):
    """
    Checks if matrix is in the positive definite cone using the Cholesky decomposition.

    Args:
    - M (np.matrix): matrix to check
    """

    try:
        _ = np.linalg.cholesky(M)
        return True
    except np.linalg.LinAlgError:
        return False

In [185]:
def project_pd(M, eps=1e-6):
    """
    Projects a matrix to the positive definite cone. If the matrix is already in the positive definite cone, 
    it is returned unchanged.

    Args:
    - M (np.matrix): matrix to project
    - eps (float): tolerance to set negative and zero eigenvalues to

    Returns:
    - M (np.matrix): projected matrix
    """
    # project a matrix to the positive definite cone
    # M: a symmetric matrix
    # return: a symmetric matrix
    if is_pd(M):
        return M
    #print("projecting")
    # eigen decomposition
    eig_vals, eig_vecs = np.linalg.eig(M)
    # set negative eigenvalues to eps
    eig_vals[eig_vals <= 0] = eps
    # reconstruct matrix
    return eig_vecs @ np.diag(eig_vals) @ eig_vecs.T


We now implement OAIS, using the following steps for each iteration:

1. Sample $x\sim q_\theta(x)$
2. Compute $w_\theta(x) = \frac{\pi(x)}{q_\theta(x)}$
3. Estimate $(\phi, \pi_\theta^N)$ using the samples $x$ and $w_\theta(x)$
4. Update $\theta$ using the gradient descent by doing:
   1. Compute $\frac{\partial \log \rho}{\partial m}$ and update $m$
   2. Compute $\frac{\partial \log \rho}{\partial S}$ and update $S$ (project to PSD cone if needed)
   

In [189]:
def OAIS(phi, pi, q0, lr, nsamples, niter):
    """
    Implement the OAIS algorithm.

    Args:
    - phi (function): test function to integrate against
    - pi (NormalRV): target distribution
    - q0 (NormalRV): initial distribution
    - lr (float): learning rate
    - nsamples (int): number of samples to draw at each iteration
    - niter (int): number of iterations

    Returns:
    - results (list): list of the results of the integration at each iteration
    - distributions (list): list of the distributions at each iteration
    """

    results = []
    distributions = [q0]
    for _ in range(niter):
        # compute inner product
        samples = q0.sample(nsamples)
        w = pi.pdf(samples) / q0.pdf(samples) # compute w as we have access to pi
        w2 = w**2
        phi_samples = np.apply_along_axis(phi, 0, samples)
        integral = np.mean(w*phi_samples)/np.mean(w)
        results.append(integral)

        # update q0
        partial_m = partial_lq_m(samples, q0)
        update_m = -np.mean(w2 * partial_m, axis=1)
        
        partial_S = partial_lq_S(samples, q0)
        update_S = -np.mean(w2*partial_S, axis=2)
        #project_pd(q0.S - lr*update_S)
        new_S = q0.S - lr*update_S
        new_m = q0.m - lr*update_m
        q0.update_vals(new_m, new_S)
        distributions.append(q0)

    return results, distributions


We now test OAIS by using the test function $\phi(x) = 1_{[-0.5,\,0.5]^2}(x)$. As our target distribution is a standard bivariate normal, the marginals of $\pi(x)$ are uncorrelated standard normal distributions; this means that, if $x\in \mathbb{R}^2$ and $y\in \mathbb{R}$ where $y\sim \mathcal{N}(0,1)$:

$$
\mathbb{P}(x\in [-0.5,\,0.5]^2) = \mathbb{P}(x_1\in [-0.5,\,0.5])\mathbb{P}(x_2\in [-0.5,\,0.5]) = [\mathbb{P}(y\leq 0.5) - \mathbb{P}(y\leq -0.5)]^2 = [\Phi(0.5) - \Phi(-0.5)]^2
$$

Therefore we can estimate the accuracy of our algorithm by computing:
$$
|(\phi, \pi^N_\theta) - [\Phi(0.5) - \Phi(-0.5)]^2|
$$

In [192]:
q = NormalRV(np.array([1,1]), np.array([[5,1],[1,5]]))
pi = NormalRV(np.array([0,0]), np.array([[1,0],[0,1]]))

def phi(x):
    # indicator function for square [-0.5, 0.5]^2
    if np.abs(x[0]) < 0.5 and np.abs(x[1]) < 0.5:
        return 1
    else:
        return 0
e,d = OAIS(phi, pi, q, 0.1, 1000, 1000)

ground_truth = sp.stats.norm.cdf(0.5) - sp.stats.norm.cdf(-0.5)
np.abs(e - ground_truth**2)


array([1.03421866e-02, 1.74237963e-04, 1.31162185e-02, 9.20361983e-03,
       5.08557975e-03, 2.78792916e-03, 8.17557279e-03, 1.37456029e-02,
       1.18860825e-02, 1.12444893e-02, 1.33523818e-02, 6.08709418e-03,
       7.62283175e-03, 1.80179984e-03, 2.16191597e-02, 1.31286363e-02,
       1.60504399e-02, 5.10850001e-03, 2.21811143e-02, 9.27137205e-03,
       1.02943468e-02, 1.02483892e-02, 1.83138551e-02, 3.65645860e-03,
       6.29797421e-03, 7.71743093e-03, 7.89892990e-03, 8.66895599e-03,
       5.52973084e-03, 8.68696797e-03, 4.39448265e-04, 2.73362653e-03,
       4.52771433e-03, 2.46065463e-03, 9.40248903e-03, 9.31024201e-03,
       5.31600427e-03, 1.12435495e-02, 6.55178916e-03, 4.29006735e-04,
       7.49271381e-03, 7.94046582e-04, 7.14816671e-03, 5.90383240e-03,
       2.14272043e-03, 3.44269297e-02, 1.99727860e-02, 1.22407758e-02,
       1.60926643e-02, 1.07531949e-03, 2.22338396e-02, 1.31417757e-02,
       1.11760504e-02, 1.00656533e-03, 8.37117942e-03, 1.33512493e-02,
      

In [182]:
d[-1].Sigma-pi.Sigma

array([[ 1.00203741, -0.00204672],
       [-0.00204672,  0.99730644]])